In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import ndimage
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from IPython.display import clear_output

import keras
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, Sequential
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img  
from keras.utils.np_utils import to_categorical

from constants import *

Using TensorFlow backend.


In [2]:
def generator(datagen, data_dir, class_mode):
    return datagen.flow_from_directory(
        data_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode=class_mode,
        shuffle=False,
        seed=0,
    )

datagen = ImageDataGenerator(rescale=1./255)

train_generator = generator(datagen, train_data_dir, 'categorical')
dev_generator = generator(datagen, dev_data_dir, 'categorical')

Found 9144 images belonging to 120 classes.
Found 1078 images belonging to 120 classes.


In [3]:
train_labels = to_categorical(train_generator.classes)
dev_labels = to_categorical(dev_generator.classes)
np.save('./data/bottleneck_features/train_labels.npy', train_labels)
np.save('./data/bottleneck_features/dev_labels.npy', dev_labels)

In [4]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3))

58892288/58889256 [==============================] - 37s 1us/step


In [5]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 112, 112, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 112, 112, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 112, 112, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 56, 56, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 56, 56, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 56, 56, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 28, 28, 128)       0         
__________

In [6]:
bottleneck_features_train = base_model.predict_generator(train_generator, len(train_generator), verbose=1)
np.save('./data/bottleneck_features/bottleneck_features_train.npy', bottleneck_features_train)

1143/1143 [==============================] - 1100s 962ms/step


In [7]:
bottleneck_features_dev = base_model.predict_generator(dev_generator, len(dev_generator), verbose=1)
np.save('./data/bottleneck_features/bottleneck_features_dev.npy', bottleneck_features_dev)

135/135 [==============================] - 128s 950ms/step
